In [1]:
!pip install unsloth datasets accelerate bitsandbytes
!pip install trl peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 2.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 24.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.8/375.8 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 52.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 1.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 MB 9.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 3.6 MB/s eta 0:00:000:00:0

In [2]:
from unsloth import FastLanguageModel
from transformers import TrainingArguments

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-bnb-4bit",     # or t5-base
    max_seq_length = 512,
    dtype = None,                        # Automatically selects bf16, fp16, or fp32
    load_in_4bit = True                  # Loads model in 4-bit for efficiency
)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-06-23 14:58:24.930444: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750690705.149256      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750690705.210214      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.6.5: Fast Mistral patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/155 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

In [3]:
from datasets import load_dataset
import json
import os

def save_qa_pairs(output_file="data/qa_pairs.json"):
    os.makedirs("data", exist_ok=True)
    dataset = load_dataset("b-mc2/cli-commands-explained", split="train")
    qa_pairs = []
    for row in dataset:
        if row['code'] and row['description']:
            qa_pairs.append({
                "question": row['description'].strip(),
                "answer": f"{row['code'].strip()}"
            })
        if len(qa_pairs) >= 200:  # Grab 200 examples
            break
    with open(output_file, "w") as f:
        json.dump(qa_pairs, f, indent=2)

if __name__ == "__main__":
    save_qa_pairs()

README.md:   0%|          | 0.00/3.78k [00:00<?, ?B/s]

cli_commands_explained.json:   0%|          | 0.00/11.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16098 [00:00<?, ? examples/s]

In [4]:
import json
import pandas as pd
from datasets import Dataset

# Load JSON
with open("/kaggle/working/data/qa_pairs.json", "r") as f:
    data = json.load(f)

df = pd.DataFrame(data)

# Add task prefix (T5 expects it)
df["input_text"] = "question: " + df["question"]
df["target_text"] = df["answer"]

# Convert to Hugging Face dataset
dataset = Dataset.from_pandas(df[["input_text", "target_text"]])


In [6]:
df

,question,answer,input_text,target_text
0,Useful when you forget to use sudo for a comma...,sudo !!,question: Useful when you forget to use sudo f...,sudo !!
1,"This Python command, using the module SimpleHT...",python -m SimpleHTTPServer,"question: This Python command, using the modul...",python -m SimpleHTTPServer
2,Really useful for when you have a typo in a pr...,^foo^bar,question: Really useful for when you have a ty...,^foo^bar
3,"Next time you are using your shell, try typing...",ctrl-x e,"question: Next time you are using your shell, ...",ctrl-x e
4,"When typing out long arguments, such as:\ncp f...",'ALT+.' or '<ESC> .',"question: When typing out long arguments, such...",'ALT+.' or '<ESC> .'
...,...,...,...,...
195,"instead of writing:\nif [[ ""$1"" == ""$2"" ]]; th...",[[ test_condition ]] && if_true_do_this || oth...,"question: instead of writing:\nif [[ ""$1"" == ""...",[[ test_condition ]] && if_true_do_this || oth...
196,This command will tell lynx to read keystrokes...,lynx -accept_all_cookies -cmd_script=/your/key...,question: This command will tell lynx to read ...,lynx -accept_all_cookies -cmd_script=/your/key...
197,Show Sample Output,bash -x ./post_to_commandlinefu.sh,question: Show Sample Output,bash -x ./post_to_commandlinefu.sh
198,This complex terminal command provides an over...,netstat -ant | awk '{print $NF}' | grep -v '[a...,question: This complex terminal command provid...,netstat -ant | awk '{print $NF}' | grep -v '[a...


In [7]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing=True,
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

Unsloth 2025.6.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [8]:

def formatting_func(example):
    return {"text": example['target_text']}

dataset = dataset.map(formatting_func)

print(dataset[0])

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

{'input_text': 'question: Useful when you forget to use sudo for a command.  "!!" grabs the last run command.', 'target_text': 'sudo !!', 'text': 'sudo !!'}


In [9]:
!pip install -q trl

In [10]:
from trl import SFTTrainer
from transformers import TrainingArguments
import torch
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 512,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 0,
        max_steps = 10,
        learning_rate = 0.0005,
        fp16 =  not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",
        group_by_length=True,
        report_to="tensorboard",
        seed = 3407,
        output_dir = "outputs",
        
    ),
)



Unsloth: Tokenizing ["text"]:   0%|          | 0/200 [00:00<?, ? examples/s]

In [11]:
# Training 
trainer_stats=trainer.train()  

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 200 | Num Epochs = 1 | Total steps = 10
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 41,943,040/7,000,000,000 (0.60% trained)


Step,Training Loss
1,1.755000
2,2.892300
3,4.471000
4,1.660400
5,2.834400
6,4.386200
7,1.757500
8,3.017100
9,3.770900
10,1.489900


Unsloth: Will smartly offload gradients to save VRAM!


In [12]:
trainer.model.save_pretrained("mistral_finetuned")
tokenizer.save_pretrained("mistral_finetuned")

('mistral_finetuned/tokenizer_config.json',
 'mistral_finetuned/special_tokens_map.json',
 'mistral_finetuned/tokenizer.model',
 'mistral_finetuned/added_tokens.json',
 'mistral_finetuned/tokenizer.json')